## Test RMSE from Analytics Vidhya  = 2534.2381

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
import statsmodels.api as stm
import seaborn as sns

In [3]:
bf = pd.read_csv('train.csv')
bf_test = pd.read_csv('test.csv')
bf.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [4]:
bf4 = bf.copy()

In [5]:
bf4.head(1)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370


In [6]:
bf4.Product_Category_2[bf4.Product_Category_2.isnull()] = 0

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
bf4.Product_Category_3[bf4.Product_Category_3.isnull()] = 0

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
bf4 = bf4.merge(pd.DataFrame(bf4.groupby(['Product_ID']).sum()['Purchase']/bf4.groupby(['Product_ID']).count()['Purchase']),how='left',left_on=['Product_ID'],right_on = ['Product_ID'])

In [9]:
bf4  = bf4.rename(columns = {'Purchase_x':'Purchase','Purchase_y':'Prod_mean'})

In [10]:
bf4['buy_price_patter'] = np.where(bf4.Purchase>bf4.Prod_mean,1,0)

In [11]:
bf4 = bf4.merge(bf4[['User_ID','buy_price_patter']].groupby('User_ID').mean().reset_index(),how='left',left_on=['User_ID'],right_on = ['User_ID'])

In [12]:
bf4  = bf4.rename(columns = {'buy_price_patter_y':'buy_price_pattern'})

In [13]:
bf5 = bf4

In [14]:
bf4 = bf5

In [15]:
bf4  = bf4.drop(['User_ID','Purchase','Product_ID','buy_price_patter_x','Stay_In_Current_City_Years','Marital_Status'],axis=1)

In [16]:
bf4.head(1)

,Gender,Age,Occupation,City_Category,Product_Category_1,Product_Category_2,Product_Category_3,Prod_mean,buy_price_pattern
0,F,0-17,10,A,3,0.0,0.0,11870.863436,0.542857


In [17]:
#City Category Dummy variable
city = pd.get_dummies(bf4.City_Category,prefix='City')
city = city.iloc[:,:-1]

In [18]:
#Gender Dummy
gender = pd.get_dummies(bf4.Gender,prefix= 'Gender')
gender = gender.iloc[:,:-1]

In [19]:
bf4.Age = bf4.Age.str.strip('+')
age = pd.get_dummies(bf4.Age,prefix= 'Age')
age = age.iloc[:,:-1]

In [20]:
x_train = pd.concat([gender,age,bf4.Occupation,city,bf4.iloc[:,4:]],axis = 1)

In [21]:
x_train = stm.add_constant(x_train)

In [22]:
y_train = bf.Purchase
x_train.head(1)

,const,Gender_F,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Occupation,City_A,City_B,Product_Category_1,Product_Category_2,Product_Category_3,Prod_mean,buy_price_pattern
0,1.0,1,1,0,0,0,0,0,10,1,0,3,0.0,0.0,11870.863436,0.542857


In [23]:
model = stm.OLS(y_train,x_train.astype(float)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Purchase   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                 1.135e+05
Date:                Wed, 15 May 2019   Prob (F-statistic):               0.00
Time:                        01:37:43   Log-Likelihood:            -5.0802e+06
No. Observations:              550068   AIC:                         1.016e+07
Df Residuals:                  550052   BIC:                         1.016e+07
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -2949.2251     24.719   -119.309      0.000   -2997.674   -2900.776
Gender_F             -81.6128      7.852    -10.394      0.000     -97.002     -66.223
Age_0-17              32.5049     26.405      1.231      0.218     -19.248      84.258
Age_18-25             26.6084     18.868      1.410      0.158     -10.373      63.590
Age_26-35             19.2670     17.950      1.073      0.283     -15.915      54.449
Age_36-45             41.5100     18.621      2.229      0.026       5.013      78.007
Age_46-50             66.5084     20.605      3.228      0.001      26.124     106.893
Age_51-55             58.1832     21.204      2.744      0.006      16.623      99.743
Occupation             0.9323      0.521      1.791      0.073      -0.088       1.953
City_A               -53.7098      9.045     -5.938      0.000     -71.437     -35.983
City_B               -15.4872      8.033     -1.928      0.054     -31.231       0.257
Product_Category_1     2.2995      0.974      2.361      0.018       0.390       4.209
Product_Category_2    -0.9310      0.541     -1.719      0.086      -1.992       0.130
Product_Category_3    -1.5533      0.596     -2.606      0.009      -2.721      -0.385
Prod_mean              0.9973      0.001   1131.766      0.000       0.996       0.999
buy_price_pattern   5537.4348     20.265    273.245      0.000    5497.715    5577.155
==============================================================================
Omnibus:                    51038.868   Durbin-Watson:                   1.923
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           122218.961
Skew:                          -0.564   Prob(JB):                         0.00
Kurtosis:                       5.015   Cond. No.                     1.43e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
bf_test.head(1)
bf4 = bf5
test = bf_test

In [25]:
test = test.merge(pd.DataFrame(bf4.groupby(['Product_ID']).sum()['Purchase']/bf4.groupby(['Product_ID']).count()['Purchase']),how='left',left_on=['Product_ID'],right_on = ['Product_ID'])

In [26]:
test  = test.rename(columns = {'Purchase':'Prod_mean'})

In [27]:
bf4['buy_price_patter'] = np.where(bf4.Purchase>bf4.Prod_mean,1,0)

In [28]:
test = test.merge(bf4[['User_ID','buy_price_patter']].groupby('User_ID').mean().reset_index(),how ='left',left_on=['User_ID'],right_on = ['User_ID'])

In [29]:
test  = test.rename(columns = {'buy_price_patter':'buy_price_pattern'})

In [30]:
test.Product_Category_2[test.Product_Category_2.isnull()]= 0
test.Product_Category_3[test.Product_Category_3.isnull()]= 0

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
test.Prod_mean[test.Prod_mean.isnull()] = test.Prod_mean.mean()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
#City Category Dummy variable
city = pd.get_dummies(test.City_Category,prefix='City')
city = city.iloc[:,:-1]

In [33]:
#Gender Dummy
gender = pd.get_dummies(test.Gender,prefix= 'Gender')
gender = gender.iloc[:,:-1]

In [34]:
test.Age = test.Age.str.strip('+')
age = pd.get_dummies(test.Age,prefix= 'Age')
age = age.iloc[:,:-1]

In [35]:
test.head(1)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Prod_mean,buy_price_pattern
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,0.0,15781.11859,0.714286


In [36]:
test1 = test

In [37]:
test = test.drop(['User_ID','Product_ID','Stay_In_Current_City_Years','Marital_Status'],axis=1)

In [38]:
x_test = pd.concat([gender,age,test.Occupation,city,test.iloc[:,4:]],axis = 1)

In [39]:
x_test=stm.add_constant(x_test)
x_test.head(1)

,const,Gender_F,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Occupation,City_A,City_B,Product_Category_1,Product_Category_2,Product_Category_3,Prod_mean,buy_price_pattern
0,1.0,0,0,0,0,0,1,0,7,0,1,1,11.0,0.0,15781.11859,0.714286


In [40]:
x_train.head(1)

,const,Gender_F,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Occupation,City_A,City_B,Product_Category_1,Product_Category_2,Product_Category_3,Prod_mean,buy_price_pattern
0,1.0,1,1,0,0,0,0,0,10,1,0,3,0.0,0.0,11870.863436,0.542857


In [41]:
y_pred = model.predict(x_test)

In [42]:
bf2 = pd.DataFrame(bf_test.User_ID,columns = ['User_ID'])
bf2['Product_ID'] = bf_test.Product_ID
bf2['Purchase'] = y_pred
bf2.head()
(bf2.isnull()).sum(axis=0)

User_ID       0
Product_ID    0
Purchase      0
dtype: int64

In [43]:
export_csv = bf2.to_csv(r"C:\Users\user\Documents\Python Scripts\ML1_assignment\Black Friday sales prediction\Sample_Submission_Tm9Lura_1.csv",header = True)

In [44]:
y_pred_train = model.predict(x_train)

In [45]:
rss_fit = sum((y_train-y_pred_train)**2)
rss_fit

3387989920190.8384

In [46]:
train_rmse = (rss_fit/x_train.shape[0])**0.5
train_rmse

2481.777622892442

In [47]:
rss_mean = sum((y_train-y_train.mean())**2)
rss_mean

13878842762287.791

In [48]:
rsqr = (rss_mean - rss_fit)/rss_mean
rsqr

0.7558881544939153